# MNIST Image Classification Example
PyTorch Example: https://github.com/pytorch/examples/blob/main/mnist/main.py

In [ ]:
!pip3 install torch torchvision torchmetrics
!pip3 install wandb[sweeps]

In [ ]:
from __future__ import print_function
from IPython.display import display
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torchmetrics import Accuracy
from pathlib import Path
import wandb

In [ ]:
SEED = 123
torch.manual_seed(SEED)
use_cuda = False
use_mps = True

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

## Prepare data

In [ ]:
dataset1 = datasets.MNIST('../data', train=True, download=True)
dataset2 = datasets.MNIST('../data', train=False)

In [ ]:
im_idxs = np.random.RandomState(123).choice(np.arange(len(dataset1)), 10)
for im_idx in im_idxs:
    im = dataset1[im_idx][0]
    display(im)

In [ ]:
BATCH_SIZE = 128

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(dataset2, batch_size=BATCH_SIZE)

## Define NN

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

## Define training loop

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 40 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

With wandb

In [ ]:
def train_wandb(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 40 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct = pred.eq(target.view_as(pred)).sum().item()
            acc = 100. * correct / BATCH_SIZE         
            wandb.log({"train_loss": loss, 
                       "train_accuracy": acc, 
                       "epoch": epoch + (batch_idx / len(train_loader))})

## Define test loop

In [ ]:
def test(model, device, val_loader):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)

    print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))

With wandb

In [ ]:
def test_wandb(model, device, val_loader, epoch):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)
    acc = 100. * correct / len(val_loader.dataset)
    print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    wandb.log({"val_loss": val_loss, "val_accuracy": acc, "epoch": epoch})

## Train model

In [ ]:
config = {
    "batch_size": BATCH_SIZE,
    "lr": 1e-4,
    "epochs": 5,
    "seed": 123,
}

In [ ]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=config["lr"])

for epoch in range(1, config["epochs"] + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, val_loader)

torch.save(model.state_dict(), "mnist_cnn.pt")

with wandb

In [ ]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=config["lr"])

Path("./mnist_cnn").mkdir(exist_ok=True)

run = wandb.init(
    project="mnist-wandb-example",
    notes="DSPRO2 Example",
    config=config,
    reinit=True,
)

torch.manual_seed(SEED)
for epoch in range(1, config["epochs"] + 1):
    train_wandb(model, device, train_loader, optimizer, epoch)
    test_wandb(model, device, val_loader, epoch)
    wandb.log({"epoch": epoch})

torch.save(model.state_dict(), "./mnist_cnn/mnist_cnn.pt")
run.log_model(path="./mnist_cnn", name="mnist-cnn")

run.finish()